## TIRCP DRISI
* DRISI emailed a request asking for TIRCP outcomes for cycles 3-5.  
* [Cycles 1-6](https://calsta.ca.gov/subject-areas/transit-intercity-rail-capital-prog)
* Cycle 1: 2015
* Cycle 2: 2016
* Cycle 3: 2018
* Cycle 4: 2020
* Cycle 5: 2022
* Cycle 6: 2023

<b>Notes 2/7/23</b>
* Application is different than what we have in the dataset.
* Outputs: hydrogen-battery fuel cell.
* DRISI wants the data before end of this week.
* Outputs
    * Category for outputs: transit/multimodal or rail/multimodal. 
    * Pull it from `allocations` tab.
    * Filter out design categories. We only care about 'CONST'.
    * Transit/Multimodal
        * Buses breakout between mobile/school/etc. Unit is each.
            * Microtransit
            * Bus Conversion
            * Zero emmission (don't distinguish between hydrogen vs battery)
            * Battery Electric
            * Ferries
    * Rail/Multimodal
        * Trolleys
        * Trains/Cars/Coach/Rolling Stocks
        * Traffic Control
    * Track
        * Track Extension
        * Double Track
    * Network Integration improves multimodal network so everything is synced. 
    * Parking Lots/Parking Deck
    * Active Transportation
        * Bike Shelters/Shade Structures
        * Bike and Pedestrians Improvements
    * Facilities
        * Center/Facility/Station/Station/Hub/Islands
    * Charging Infrastructure. 
    * Storm Drain Line
    * Street Extension
    * Charging
    * Signaling
* Outcomes
    * On Time Performance
        * Expanded Service
        * Routes 
        * Ridership

In [ ]:
import itertools

import A1_data_prep
import A2_tableau
import A8_strings
import numpy as np
import pandas as pd
from babel.numbers import format_currency
from calitp import *

In [ ]:
pd.options.display.max_columns = 125
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
# GCS File Path:
GCS_FILE_PATH = f"{A1_data_prep.GCS_FILE_PATH}DRISI/"

In [ ]:
GCS_FILE_PATH

## Prep Functions

In [ ]:
def read_in_files(file_name: str, columns_to_drop: list):
    df = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}{file_name}"))

    df = df.drop(columns=columns_to_drop)
    
    # Delete any columns with the following strings
    df = df.loc[:, ~df.columns.str.contains("funding|recipient|date|cost|program|amount|name|description")]
    
    df = df.fillna(df.dtypes.replace({"float64": 0.0, "object": "None"}))
    
    return df

In [ ]:
def merge_value_counts(left_df, right_df, left_m_col:str, right_m_col:str):
    # Some of these sheets have repeated columns
    # Del them off so the info isn't repeated again
    left_df_cols = left_df.columns
    right_df_cols = right_df.columns
    common_cols = right_df_cols.intersection(left_df_cols)
    common_cols = common_cols.to_list()
    
    right_df = right_df.drop(columns = common_cols) 
    
    # One df for an outer merge just for checking both/left only/right only vals
    outer = pd.merge(left_df, right_df, how="outer", left_on=left_m_col, right_on=right_m_col, indicator=True)
    
    # One for left
    left =  pd.merge(left_df, right_df, how="left",  left_on=left_m_col, right_on=right_m_col)

    return outer, left

## Outputs
### Manipulate TIRCP Project Sheet

In [ ]:
allocation_cols = [
    "allocation_ppno",
    "allocation_award_year",
    "allocation_components",
    "allocation_phase",
]
project_cols = [
    "project_ppno",
    "project_award_year",
    "project_project_#",
    "project_project_title",
    "project_project_description",
    "project_grant_recipient",
]

In [ ]:
# project2.groupby('project_award_year').size(), project2.groupby('project_award_year').project_ppno.nunique()

In [ ]:
# Read in a joined allocation-project sheet
tircp = A1_data_prep.merge_allocation_project(project_cols, allocation_cols, "left")

In [ ]:
# Drop irrelevant years & non-construction.
tircp2 = tircp[(tircp["project_award_year"] >= 2018)].reset_index(drop=True)

In [ ]:
# Double check that the PPNO are correct
tircp2.groupby("project_award_year").project_ppno.nunique()

In [ ]:
# Sort df by award year, number, and phase.
tircp2 = tircp2.sort_values(
    ["project_award_year", "project_project_#", "allocation_phase"]
)

In [ ]:
# Create project number
# Map 0 to another single digit numbers
# https://stackoverflow.com/questions/20990863/python-pandas-add-leading-zero-to-make-all-months-2-digits
tircp2["project_number"] = (
    tircp2["project_award_year"].astype(str)
    + "-"
    + tircp2["project_project_#"].astype(str)
)

In [ ]:
tircp2.allocation_phase.value_counts()

In [ ]:
len(tircp2), len(
    tircp2.drop_duplicates(subset=["project_ppno", "allocation_components"])
)

In [ ]:
# Drop duplicates by ppno and allocation componentes
# Sorted by allocation phase, so construction is kept
# if components is the same across multiple phases
tircp3 = (
    tircp2.drop_duplicates(subset=["project_ppno", "allocation_components"])
).reset_index(drop=True)

In [ ]:
tircp3.groupby("project_award_year").project_ppno.nunique()

In [ ]:
# For projects that don't have any allocation components info
# fill it in with project description
# Cp068 & cp106
tircp3.allocation_components = tircp3.allocation_components.fillna(
    tircp3.project_project_description
)

### Extract Outputs

In [ ]:
tircp3 = A8_strings.simplify_descriptions(
    tircp3,
    "allocation_components",
    "clean_components",
    A8_strings.description_words_to_delete,
)

In [ ]:
tircp3 = tircp3.drop(columns=["allocation_ppno", "allocation_award_year", "_merge"])

In [ ]:
track = ["track", "double track"]
rail = ["trolley", "train", "car", "coach", "rolling", "traffic", "light rail", "rail"]
bus = ["bus", "van", "buses", "microtransit", "micro transit", "zebs"]
ferry = ["ferry", "ferries", "vessel"]
active_transportation = [
    "bike",
    "shelter",
    "pedestrian",
    "cycle",
    "crosswalk",
    "sidewalk",
    "bicycle",
]
network = ["network"]
parking = ["parking", "lots", "deck"]
facilities = [
    "center",
    "facility",
    "station",
    "hub",
    "islands",
    "shelter",
    "shade",
    "location",
    "center",
    "stations"
]
charging = ["charging", "charge"]
storm_drain = ["storm"]
street = [
    "street",
    "signal",
    "signaling",
    "traffic control",
    "road",
    "lane",
    "surface",
    "interchange",
    "intersection",
    "shoulder",
    "grade separations",
    "crossing",
    "border",
    "corridor",
]
other_vehicles = ["vehicle", "zemu"]
bridge_tunnel = ["bridge", "tunnel"]
infrastructure = [
    "infrastructure",
    "bluff",
    "operation",
    "ramp",
    "mainline",
    "port",
    "fiber optic",
]
ontime_transit_improvements = [
    "ITS",
    "signange",
    "mobile app",
    "ticket",
    "ridership",
    "expanded",
    "service",
    "time",
    "route",
]

In [ ]:
my_keywords_list = [
    track,
    rail,
    bus,
    ferry,
    active_transportation,
    network,
    parking,
    facilities,
    charging,
    storm_drain,
    street,
    other_vehicles,
    bridge_tunnel,
    infrastructure,
    ontime_transit_improvements,
]

In [ ]:
my_new_column_names_list = [
    "track",
    "rail",
    "bus",
    "ferry",
    "active_transportation",
    "network",
    "parking",
    "facilities",
    "charging",
    "storm_drain",
    "street",
    "other_vehicles",
    "bridge_tunnels",
    "infrastructure",
    "ontime_transit_improvements",
]

In [ ]:
tircp3 = A8_strings.total_procurement_estimates(
    tircp3,
    "clean_components",
    my_keywords_list,
    my_new_column_names_list,
)

In [ ]:
def fill_in_zeroes(df, keywords: list, description_column: str, new_col_name: str):
    # Delinate items in keywords list using |
    keywords_blob = f"({'|'.join(keywords)})"

    # If a keyword appears in the desc
    # Automatically add it as 1
    keywords_dict = dict.fromkeys(keywords, 1)

    df[new_col_name] = (
        df[description_column]
        .str.extract(keywords_blob, expand=False)
        .replace(keywords_dict)
        .fillna(0)
    )
    return df

In [ ]:
# Turn this into a function later
for i in range(0, len(my_keywords_list)):
    tircp4 = fill_in_zeroes(
        tircp3,
        my_keywords_list[i],
        "clean_components",
        f"new_{my_new_column_names_list[i]}",
    )

    # Replace any zeroes in the original columns with 1
    # if a keyword is found
    # https://stackoverflow.com/questions/68243146/replace-zero-with-value-of-an-other-column-using-pandas
    tircp4[f"total_{my_new_column_names_list[i]}"] = tircp4[
        f"total_{my_new_column_names_list[i]}"
    ].mask(
        tircp4[f"total_{my_new_column_names_list[i]}"].eq(0),
        tircp4[f"new_{my_new_column_names_list[i]}"],
    )

    tircp4 = tircp4[tircp4.columns.drop(list(tircp4.filter(regex="new")))]

In [ ]:
additional_keywords = [
    "microtransit",
    "emission",
    "conversion",
    "zero",
    "hydrogen",
    "battery",
    "electric",
    "hybrid",
    "zev",
    "zemu",
]

In [ ]:
def extract_keywords(df, list_of_words: list, more_keywords: list, column: str):
    """
    Extract keywords found in a certain column
    into a new column called "categories"
    """
    for i in list_of_words: more_keywords.extend(i)

    query = "|".join(more_keywords)

    df["categories"] = df[column].str.lower().str.findall(r"\b({})\b".format(query))
    return df

In [ ]:
tircp4 = extract_keywords(
    tircp4, my_keywords_list, additional_keywords, "allocation_components"
)

### Add in info from TIRCP Battery and Hydrogen Fuel Cell Bus list 10-10-2022 (1).xlsx

In [ ]:
battery_drop_cols = [
    "local_agency_contact",
    "local_agency_email",
    "local_agency_phone_number",
    "awarded_allocated",
    "procured_contracted",
    "components",
]

In [ ]:
battery = read_in_files("TIRCP Battery and Hydrogen Fuel Cell Bus list 10-10-2022 (1).xlsx", battery_drop_cols)

In [ ]:
battery.sample()

In [ ]:
battery["project_number"] = (
    battery["award_year"].astype(str) + "-" + battery["project_#"].map("{:02}".format).astype(str)
)

In [ ]:
battery["hydrogen_battery_buses"] = battery['#_hydrogen_fuel_cell_buses'] + battery['#_battery_electric_buses']

In [ ]:
tircp4.merge(battery[['project_number','hydrogen_battery_buses']], how="outer", on=["project_number"], indicator= True)[["_merge"]].value_counts()

In [ ]:
tircp4 = tircp4.merge(battery[['project_number','hydrogen_battery_buses']], how="left", on=["project_number"])

In [ ]:
tircp4["total_bus"] = tircp4["total_bus"].mask(
        tircp4["total_bus"].eq(0),
        tircp4["hydrogen_battery_buses"],
    ).fillna(0)

In [ ]:
# tircp4[['project_number','total_bus','hydrogen_battery_buses']]

### Clean

In [ ]:
# Subset for only relevant columns
outputs_cols = [
    "project_project_title",
    "project_award_year",
    "allocation_components",
    "categories",
    "total_track",
    "total_rail",
    "total_bus",
    "total_ferry",
    "total_active_transportation",
    "total_network",
    "total_parking",
    "total_facilities",
    "total_charging",
    "total_storm_drain",
    "total_street",
    "total_other_vehicles",
    "total_bridge_tunnels",
    "total_infrastructure",
    "total_ontime_transit_improvements",
]

In [ ]:
outputs = tircp4[outputs_cols]

In [ ]:
groupby_cols = [
    "project_project_title",
    "project_award_year",
    "allocation_components",
    "total_track",
    "total_rail",
    "total_bus",
    "total_ferry",
    "total_active_transportation",
    "total_network",
    "total_parking",
    "total_facilities",
    "total_charging",
    "total_storm_drain",
    "total_street",
    "total_other_vehicles",
    "total_bridge_tunnels",
    "total_infrastructure",
    "total_ontime_transit_improvements",
]

In [ ]:
# All the category values are packed into a list
# Have to expode them out in order to use groupby
# to do: add this step in the function 'extract_keywords'
outputs2 = (
    outputs.explode("categories")
    .sort_values(["project_award_year", "project_project_title"])
    .drop_duplicates(subset=["project_project_title", "categories"])
    .fillna(" ")
)

In [ ]:
outputs2 = outputs2.groupby(groupby_cols)["categories"].apply(",".join).reset_index()

In [ ]:
agg_cols = [
    "total_bus",
    "total_ferry",
    "total_active_transportation",
    "total_network",
    "total_parking",
    "total_facilities",
    "total_charging",
    "total_storm_drain",
    "total_street",
    "total_other_vehicles",
    "total_bridge_tunnels",
    "total_infrastructure",
    "total_ontime_transit_improvements",
]

In [ ]:
outputs3 = outputs2.groupby([
        "project_award_year",
        "project_project_title",
        "allocation_components",
        "categories",
    ]
).agg({**{e: "max" for e in agg_cols}})

## Outcomes
### Clean up Projects Sheet

In [ ]:
def clean_project():
    project = A1_data_prep.clean_project()
    
    project = project.loc[project["project_award_year"] >=2018].reset_index(drop = True)
    
    project["project_number"] = (project["project_award_year"].astype(str) 
                                 + "-"+ project["project_project_#"].map("{:02}".format).astype(str))
    
    project = project[['project_award_year','project_project_title','project_grant_recipient','project_ppno', 'project_number']]
    
    return project

In [ ]:
# Subset tircp
project = clean_project()

In [ ]:
project.shape

### AwardedProjectsDetail.xlsx

In [ ]:
drisi_drop_cols = [
    "agency_code",
    "agency_short_name",
    "sub_program_description",
    "agency_name",
    "program_short_name",
    "program_name",
    "program_description",
    "sub_program_short_name",
    "sub_program_name",
    "project_type",
    "agency",
    "program",
    "date_imported",
    "contractor_or_awardee_admin_expenses",
    "voucher_id",
    "project_is_completed",
    "project_is_canceled"
]

In [ ]:
drisi = read_in_files("AwardedProjectsDetail.xlsx", drisi_drop_cols)

In [ ]:
drisi.shape

In [ ]:
outer_drisi, m1 = merge_value_counts(project, drisi, "project_number", "project_id",) 

In [ ]:
m1 = m1.drop(columns = ['record_type', "project_id", "project_life_years", "project_status"])

In [ ]:
m1.shape

### ImplementedProjectsDetail.xlsx

In [ ]:
implemented_drop_cols = [
    "programuniqueidentifier",
    "record_type",
    "reporting_cycle_name",
    "agency_short_name",
    "agency_name",
    "date_operational",
    "program_name",
    "program_description",
    "project_completion_date",
    "date_imported",
    "sub_program_name",
    "date_selected_for_award",
    "project_name",
    "project_type",
    "fiscal_year_funding_project",
    "census_tract",
    "address",
    "lat_long",
    "total_program_ggrffunding",
    "voucher_name",
    "voucher_description",
]

In [ ]:
implemented = read_in_files("ImplementedProjectsDetail.xlsx", implemented_drop_cols)

In [ ]:
outer_implemented, m2 = merge_value_counts(m1, implemented, "project_number", "project_idnumber") 

In [ ]:
m2 = m2.drop(columns = ['project_count',"project_idnumber",'proj_rec_id','voucher_id'])

In [ ]:
# Why does the df become so large?
m2.shape

In [ ]:
m2 = m2.drop_duplicates(subset = ["project_ppno", "project_number"])

In [ ]:
m2.shape

### OutcomeProjectsDetail.xlsx 

In [ ]:
outcomes_drop_cols = [
    "unnamed:_0",
    "proj_rec_id",
    "reporting_cycle_name",
    "agency_short_name",
    "agency_name",
    "program_short_name",
    "program_name",
    "program_description",
    "record_type",
    "sub_program_short_name",
    "sub_program_name",
    "sub_program_description",
    "date_imported",
]

In [ ]:
outcomes = read_in_files("OutcomeProjectsDetail.xlsx", outcomes_drop_cols)

In [ ]:
# outcomes.sort_values('projectid_number')

In [ ]:
# list(outcomes.columns)

In [ ]:
outer_outcomes, m3 = merge_value_counts(m2, outcomes, "project_number", "projectid_number") 

In [ ]:
outer_outcomes._merge.value_counts()

In [ ]:
outer_outcomes.loc[outer_outcomes._merge == "right_only"]['projectid_number'].unique()

In [ ]:
m3 = m3.drop(columns = ["projectid_number"])

In [ ]:
m3.shape

In [ ]:
id_cols = [
 'project_number',
 'project_project_title']

In [ ]:
value_cols = ['annual_project_ghgreductions',
 'total_project_ghgreductions',
 'vmtreductions',
 'number_of_housing_units',
 'number_of_affordable_housing_units',
 'estimated_total_recycling_tons',
 'estimated_ridership_increases',
 'estimated_acres_preserved',
 'estimated_acres_restored',
 'estimated_num_riders_served',
 'estimated_waste_digested_tons',
 'estimated_diverted_from_landfills_tons',
 'reclaimed_food_tons',
 'estimated_incentives_to_be_issued',
 'number_of_rebates_issued',
 'estimated_number_of_trees_to_be_planted',
 'estimated_energy_saved_kwh',
 'estimated_energy_saved_therms',
 'estimated_water_saved_gallons',
 'estimated_energy_generated_kwh',
 'estimated_total_criteria_air_pol_red_tons',
 'estimated_fuel_use_reduction_gal',
 'estimated_num_vehicles_in_service',
 'cesversion',
 'direct_jobs_fte',
 'indirect_jobs_fte',
 'induced_jobs_fte',
 'non_ggrffunded',
 'project_life_years',
 'is_benefit_disadvantaged_communities',
 'disadvantaged_community_criteria',
 'disadvantaged_community_need',
 'disadvantaged_community_census_tracts',
 'vouchers_benefiting_disadvantaged_communities',
 'rebates_within_disadvantaged_communities',
 'ab1550choice',
 'buffer_count',
 'is_ab1550buffer_region',
 'dac1550count',
 'is_benefit_dac1550communities',
 'low_income_count',
 'low_income_housing_count',
 'is_low_income_communities',
 'potential_buffer_count',
 'potential_dac1550count',
 'potential_low_income_count',
 'estimated_acres_treated',
 'renewable_fuel_generation_tons',
 'wood_burning_reduction_cords',
 'black_carbon_reductions_pounds',
 'stateew_dpm',
 'stateew_nox',
 'stateew_pm25',
 'stateew_rog',
 'diesel_pm_reductions_pounds',
 'nox_reductions_pounds',
 'pm25reductions_pounds',
 'rog_reductions_pounds',
 'number_of_plans',
 'pollinator_acreage',
 'research_grant',
 'science_advancement',
 'soil_benefit',
 'est_energy_gen_scf',
 'est_source_red_tons',
 'fuel_treatment_num',
 'education_event_num',
 'energy_audit_buildings',
 'est_divert_landfills_tons',
 'est_fuel_gen_gal',
 'climate__adaptation',
 'community__engagement',
 'compost__produced_tons',
 'compost__produced_tons__yr',
 'net_density__dua',
 'applicants__assisted',
 'invasive_cover_12_months',
 'invasive_cover_36_months',
 'project_acreage',
 'intermediaryadminexpenses',
 'nongggrf_funded',
 'dactable',
 'fg17comm_need',
 'fg17comm_need_qual',
 'daccommunity_benefit_critieria_met',
 'acres_preserved',
 'acres_treated',
 'amt_reduction',
 'baseline_pasture_weeks',
 'biomass_delivered_bdtyr',
 'add_acres_preserved',
 'add_acres_preserved_1',
 'biomass_carbon_change_mtco2yr',
 'biomass_delivered_bfyr',
 'ch4em_rate_mtco2yr',
 'co2em_rate_mtco2yr',
 'cardboard_recycled_tons_yr',
 'census_tract',
 'commercial_space_use_pct',
 'compost_produced_tons_yr',
 'compost_rate_stayr',
 'cover_crop_desc',
 'dpmchange_tons_yr',
 'daily_riders',
 'days_op_yr',
 'diesel_change_gal_yr',
 'diesel_use_gal_yr',
 'disturbance_tree_mortality_rate',
 'diverted_tons_yr',
 'energy_change_kwh_hhs',
 'energy_change_kwh_yr',
 'energy_change_mmbtuhhs',
 'energy_change_scfhhs',
 'energy_change_scfry',
 'energy_gen_gal_yr',
 'energy_gen_kwh_gs',
 'energy_gen_kwh_yr',
 'energy_gen_mmbtuyr',
 'energy_gen_mwyr',
 'energy_gen_scfyr',
 'energy_use_kwh_yr',
 'energy_use_scfyr',
 'fireplace_cstove',
 'fireplace_electric',
 'fireplace_ncstove',
 'fireplace_ng',
 'fireplace_propane',
 'food_rescued_tons_yr',
 'food_source_red_tons_yr',
 'ghgchange_mtco2yr',
 'gasoline_change_gal_yr',
 'gasoline_user_gal_yr',
 'glass_recycled_tons_yr',
 'hdprecycled_tons_yr',
 'hardwood_pct',
 'harvested_wood_bdtyr',
 'harvested_wood_bfyr',
 'housing_occupancy_rate',
 'is_agreement_check',
 'low_income_mode_share',
 'lumber_recycled_tons_yr',
 'mag_mail_recycled_tons_yr',
 'mattress_recycled_tons_yr',
 'mattress_renovate_tons_yr',
 'methane_destroy_mtyr',
 'methane_used_mtyr',
 'mill_efficiency_pct',
 'misc_pct',
 'n2oem_rate_mtco2yr',
 'newspaper_recycled_tons_yr',
 'nox_change_tons_yr',
 'nyl66carpet_cush_recycled_tons_yr',
 'nyl66carpet_engr_recycled_tons_yr',
 'nyl66carpet_tile_recycled_tons_yr',
 'nyl6carpet_cush_recycled_tons_yr',
 'nyl6carpet_engr_recycled_tons_yr',
 'nyl6carpet_tile_recycled_tons_yr',
 'other_material',
 'other_material_tons_yr',
 'petcarpet_cush_recycled_tons_yr',
 'petrecycled_tons_yr',
 'pmchange_tons_yr',
 'panels_pct',
 'paper_pct',
 'phonebooks_recycled_tons_yr',
 'planted_tree_mortality_rate',
 'plywood_pct',
 'poly_carpet_cush_recycled_tons_yr',
 'poly_carpet_engr_recycled_tons_yr',
 'poly_carpet_yrn_recycled_tons_yr',
 'project_pasture_weeks',
 'project_pasture_weeks_1',
 'quantity_buildings',
 'quantity_residences',
 'rogchange_tons_yr',
 'resident_mode_share',
 'resident_transit_use_pct',
 'residual_lanfilled_mat_tons_yr',
 'ridership_day',
 'softwood_pct',
 'soil_carbon_change_mtcoyr',
 'standboard_pct',
 'storm_water_capture_afyr',
 'tech_assessment',
 'tech_assessment_1',
 'textiles_recycled_tons_yr',
 'treated_tree_mortality_rate',
 'trees_planted',
 'uncertified_cstove',
 'uncertified_electric',
 'uncertified_ncstove',
 'uncertified_ng',
 'uncertified_propane',
 'users_per_day',
 'vmtchange_year',
 'vmtyear',
 'water_use_change_mgal_yr',
 'wood_mill_bdtyr',
 'wood_mill_bfyr',
 'wood_use_chang_cord_hhs',
 'fuel_type',
 'housing_occupancy_rate_li',
 'number_of_vouchers_tracked',
 'diverted_comp_tons_yr',
 'is_cnratio',
 'publications_num',
 'successful_grantees']

In [ ]:
# Group by project name and number
m3_group1 = m3.groupby(id_cols).agg({**{e: "max" for e in value_cols}}).T.fillna(0)

### TIRCP_AllProjects_12212022 (002).xlsx
* Doesn't have anything interesting.

In [ ]:
# Fill in empty values with NA
all_projects = read_in_files("TIRCP_AllProjects_12212022 (002).xlsx", [])

In [ ]:
# all_projects.sample()

## Save

In [ ]:
with pd.ExcelWriter(f"{GCS_FILE_PATH}drisi_outcomes_outputs.xlsx") as writer:
    outputs3.to_excel(writer, sheet_name="outputs", index=True)
    m3_group1.to_excel(writer, sheet_name="outcomes", index=True)

### Save

In [ ]:
"""
with pd.ExcelWriter(f"{GCS_FILE_PATH}calsta_draft.xlsx") as writer:
    outcomes.to_excel(writer, sheet_name="outcomes_unpivoted", index=True)
    outcomes_transformed.to_excel(writer, sheet_name="outcomes_transformed", index=True)
    projects.to_excel(writer, sheet_name="projects", index=True)
    year_summary.to_excel(writer, sheet_name="year_summary", index=True)
    GHG_by_year.to_excel(writer, sheet_name="GHG_reduction_year", index=True)
    """